In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/porto-seguro-safe-driver-prediction/sample_submission.csv
/kaggle/input/porto-seguro-safe-driver-prediction/train.csv
/kaggle/input/porto-seguro-safe-driver-prediction/test.csv


In [39]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from path import Path
from sklearn.model_selection import StratifiedKFold

In [40]:

class Config:
    input_path = Path('../input/porto-seguro-safe-driver-prediction')
    optuna_lgb = False
    n_estimators = 1500
    early_stopping_round = 150
    cv_folds = 5
    random_state = 0
    params = {'objective': 'binary',
              'boosting_type': 'gbdt',
              'learning_rate': 0.01,
              'max_bin': 25,
              'num_leaves': 31,
              'min_child_samples': 1500,
              'colsample_bytree': 0.7,
              'subsample_freq': 1,
              'subsample': 0.7,
              'reg_alpha': 1.0,
              'reg_lambda': 1.0,
              'verbosity': 0,
              'random_state': 0}
    
config = Config()

In [41]:
train = pd.read_csv(config.input_path / 'train.csv', index_col='id')
test = pd.read_csv(config.input_path / 'test.csv', index_col='id')
submission = pd.read_csv(config.input_path / 'sample_submission.csv', index_col='id')
# train["target"].unique()

In [42]:
calc_features = [feat for feat in train.columns if "_calc" in feat]
cat_features = [feat for feat in train.columns if "_cat" in feat]
# cat_features

In [43]:
target = train["target"]
train = train.drop("target", axis="columns")

In [44]:
train = train.drop(calc_features, axis="columns")
test = test.drop(calc_features, axis="columns")

In [45]:
train = pd.get_dummies(train, columns=cat_features, dtype=float)

In [46]:
test = pd.get_dummies(test, columns=cat_features, dtype=float)

In [47]:
assert((train.columns==test.columns).all())

In [51]:
from numba import jit

@jit
def eval_gini(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_pred)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_lgb(y_true, y_pred):
    eval_name = 'normalized_gini_coef'
    eval_result = eval_gini(y_true, y_pred)
    is_higher_better = True
    return eval_name, eval_result, is_higher_better

/tmp/ipykernel_33/2566664522.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def eval_gini(y_true, y_pred):


In [52]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import numpy as np

if config.optuna_lgb:
    
    def objective(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
            "num_leaves": trial.suggest_int("num_leaves", 3, 255),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 3000),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
            "subsample_freq": trial.suggest_int("subsample_freq", 0, 10),
            "subsample": trial.suggest_float("subsample", 0.1, 1.0),
            "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-9, 10.0),
            "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-9, 10.0)
        }
        
        score = list()
        skf = StratifiedKFold(n_splits=config.cv_folds, shuffle=True, random_state=config.random_state)
        
        for train_idx, valid_idx in skf.split(train, target):
            X_train = train.iloc[train_idx]
            y_train = target.iloc[train_idx]
            X_valid = train.iloc[valid_idx]
            y_valid = target.iloc[valid_idx]
            
            model = lgb.LGBMClassifier(**params, n_estimators=1500, early_stopping_round=150, force_row_wise=True)
            
            callbacks = [lgb.early_stopping(stopping_rounds=150, verbose=False)]
            
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=gini_lgb, callbacks=callbacks)
            
            score.append(model.best_score_["valid_0"]["normalized_gini_coef"])
            
        return np.mean(score)
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=300)
    
    print("Best Gini Normalized Score", study.best_value)
    print("Best parameters", study.best_params)
    
    params = {
        "objective": "binary",
        "boosting_type": "gbdt",
        "verbosity": 0,
        "random_state": 0
    }
    
    params.update(study.best_params)
else:
    params = config.params


In [53]:
preds = np.zeros(len(test))
oof = np.zeros(len(train))
metric_evaluations = list()

skf = StratifiedKFold(n_splits=config.cv_folds, shuffle=True, random_state=config.random_state)

for idx, (train_idx, valid_idx) in enumerate(skf.split(train, target)):
    print(f"CV fold {idx}")
    X_train, y_train = train.iloc[train_idx], target.iloc[train_idx]
    X_valid, y_valid = train.iloc[valid_idx], target.iloc[valid_idx]
    
    model = lgb.LGBMClassifier(**params, n_estimators=config.n_estimators, early_stopping_round=config.early_stopping_round, force_row_wise=True)
    
    callbacks=[lgb.early_stopping(stopping_rounds=150),
              lgb.log_evaluation(period=100, show_stdv=False)]
    
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
             eval_metric=gini_lgb, callbacks=callbacks)
    
    metric_evaluations.append(model.best_score_["valid_0"]["normalized_gini_coef"])
    preds += (model.predict_proba(test, num_iteration=model.best_iteration_)[:, 1] / skf.n_splits)
    
    oof[valid_idx] = model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:, 1]

CV fold 0
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153243	valid_0's normalized_gini_coef: 0.271457
[200]	valid_0's binary_logloss: 0.15228	valid_0's normalized_gini_coef: 0.280599
[300]	valid_0's binary_logloss: 0.15185	valid_0's normalized_gini_coef: 0.286829
[400]	valid_0's binary_logloss: 0.151651	valid_0's normalized_gini_coef: 0.289906
[500]	valid_0's binary_logloss: 0.151543	valid_0's normalized_gini_coef: 0.291906
[600]	valid_0's binary_logloss: 0.151473	valid_0's normalized_gini_coef: 0.293377
[700]	valid_0's binary_logloss: 0.151437	valid_0's normalized_gini_coef: 0.293827
[800]	valid_0's binary_logloss: 0.151417	valid_0's normalized_gini_coef: 0.294276
[900]	valid_0's binary_logloss: 0.15142	valid_0's normalized_gini_coef: 0.294119
CV fold 1
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153553	valid_0's normalized_gini_coef: 0.255568
[200]	valid_0's binary_logloss: 0.15277

In [54]:
print(f"LightGBM CV Gini Normalized Score: {np.mean(metric_evaluations):0.3f}({np.std(metric_evaluations):0.3f})")

LightGBM CV Gini Normalized Score: 0.289(0.015)


In [ ]:
def batch_generator(X, batch_size, shuffle=True, random_state=None):
    batch_index = 0
    n = X.shape[0]
    while True:
        if batch_index == 0:
            index_array = np.arange(n)
            if shuffle:
                np.random.seed(seed=random_state)
                index_array = np.random.permutation(n)
                
        current_index = (batch_index * batch_size) % n
        if n >= current_index + batch_size:
            current_batxh_size = batch_size
            batch_index += 1
        else:
            current_batch_size = n - current_index
            batch_size = 0
            
        batch = X[index_array[current_index: current_index + current_batch_size]]
        
        yield batch

In [55]:
submission["target"] = preds
submission.to_csv("lgb_submission.csv")

oofs = target.to_frame()
oofs["target"] = oof
oofs.to_csv("lgb_oof.csv")

In [ ]:
def mixup_generator(X, batch_size, swaprate=0.15, shuffle=True, random_state=None):
    if random_state is None:
        random_state = np.randint(0, 999)
    
    num_features = X.shape[1]
    num_swaps = int(num_features * swaprate)
    generator_a = batch_generator(X, batch_size, shuffle, random_state)
    generator_b = batch_generator(X, batch_size, shuffle, random_state+1)
    
    while True:
        batch = next(generator_a)
        mixed_batch = batch.copy()
        effective_batch_size = batch.shape[0]
        alternative_batch = next(generator_b)
        assert((batch != alternative_batch).any())
        for i in range(effective_batch_size):
            swap_idx = np.random.choice(num_features, num_swaps, replace=False)
            mixed_batch[i, swap_idx] = alternative_batch[i, swap_idx]
        yield (mixed_batch, batch)

In [ ]:
def get_DAE(X, architecture=[1500, 1500, 1500]):
    features = X.shape[1]
    inputs = Input((features,))
    for i, nodes in enumerate(architecture):
        layer = Dense(nodes, activation="relu", use_bias=False, name=f"code_{i+1}")
        if i==0:
            x = layer(inputs)
        else:
            x = layer(x)
        x = BatchNormalization()(x)
    outputs = Dense(features, activation="linear")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="mse", metrics=["mse", "mae"])
    return model

In [ ]:
def extract_dae_features(autoencoder, X, layers=[3]):
    data = []
    for layer in layers:
        if layer==0:
            data.append(X)
        else:
            get_layer_output = Model([autoencoder.layers[0].input],
                                    [autoencoder.layers[layer].output])
            layer_output = get_layer_output.predict(X, batch_size=128)
            data.append(layer_output)
    data = np.hstack(data)
    return data

In [ ]:
"""
def autoencoder_fitting(X_train, X_valid, filename="dae", random_state=None, suppress_output=False):
    if suppress_output:
        verbose = 0
    else:
        verbose = 2
        print("Fittiong a denoising autoencoder")
        
    tf.random.set_seed(seed=random_state)
    generator = mixup_generator(X_train, batch_size=config.dae_batch_size, swaprate=0.15, random_state=config.random_state)
    dae = get_DAE(X_train, architecture=config.dae_architecture)
    steps_per_epoch = np.ceil(X_train.shape[0] / config.dae_batch_size)
    
    early_stopping = EarlStopping(monitor="val_mse",
                                 mode="min",
                                 patience=5,
                                 restore_best_weight=True,
                                 verbose=0)
    history = dae.fit(generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=config.dae_num_epoch,
                     )
                     """
